In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasets/bakhtiyar2222/refined-deep-sar-oil-spill-sos-dataset/images/images/val/palsar_151.png
/kaggle/input/datasets/bakhtiyar2222/refined-deep-sar-oil-spill-sos-dataset/images/images/val/palsar_679.png
/kaggle/input/datasets/bakhtiyar2222/refined-deep-sar-oil-spill-sos-dataset/images/images/val/sentinel_24.png
/kaggle/input/datasets/bakhtiyar2222/refined-deep-sar-oil-spill-sos-dataset/images/images/val/sentinel_4.png
/kaggle/input/datasets/bakhtiyar2222/refined-deep-sar-oil-spill-sos-dataset/images/images/val/palsar_371.png
/kaggle/input/datasets/bakhtiyar2222/refined-deep-sar-oil-spill-sos-dataset/images/images/val/sentinel_9.png
/kaggle/input/datasets/bakhtiyar2222/refined-deep-sar-oil-spill-sos-dataset/images/images/val/palsar_173.png
/kaggle/input/datasets/bakhtiyar2222/refined-deep-sar-oil-spill-sos-dataset/images/images/val/sentinel_513.png
/kaggle/input/datasets/bakhtiyar2222/refined-deep-sar-oil-spill-sos-dataset/images/images/val/palsar_302.png
/kaggle/input/da

In [3]:
!pip install -q segmentation-models-pytorch albumentations opencv-python

In [10]:
import os
import cv2
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2

import segmentation_models_pytorch as smp

In [25]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DATA_ROOT = "/kaggle/input/datasets/bakhtiyar2222/refined-deep-sar-oil-spill-sos-dataset"
IMG_TRAIN = f"{DATA_ROOT}/images/images/train"
IMG_VAL   = f"{DATA_ROOT}/images/images/val"

MASK_TRAIN = f"{DATA_ROOT}/masks/masks/train"
MASK_VAL   = f"{DATA_ROOT}/masks/masks/val"

BATCH_SIZE = 8
LR = 2e-4
EPOCHS = 80

In [26]:
class OilSpillDataset(Dataset):
    def __init__(self, img_dir, mask_dir, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.images = sorted(os.listdir(img_dir))
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx])

        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        mask  = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        image = image.astype(np.float32)
        image = (image - image.min()) / (image.max() - image.min() + 1e-6)

        mask = (mask > 0).astype(np.float32)

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented["image"]
            mask = augmented["mask"]

        return image, mask.unsqueeze(0)

In [27]:

train_tfms = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.GaussNoise(
        std_range=(0.02, 0.1),
        mean_range=(0.0, 0.0),
        p=0.2
    ),
    ToTensorV2(),
])

val_tfms = A.Compose([
    ToTensorV2(),
])

In [28]:
train_ds = OilSpillDataset(IMG_TRAIN, MASK_TRAIN, train_tfms)
val_ds   = OilSpillDataset(IMG_VAL, MASK_VAL, val_tfms)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

In [29]:
model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",  
    in_channels=1,
    classes=1,
)

model = model.to(DEVICE)

config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

In [30]:
bce = nn.BCEWithLogitsLoss()
dice = smp.losses.DiceLoss(mode="binary")

def loss_fn(pred, target):
    return 0.5 * bce(pred, target) + 0.5 * dice(pred, target)

optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [31]:
def iou_score(pred, target):
    pred = torch.sigmoid(pred)
    pred = (pred > 0.5).float()

    intersection = (pred * target).sum()
    union = pred.sum() + target.sum() - intersection

    return (intersection + 1e-6) / (union + 1e-6)

In [32]:
best_iou = 0

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0

    for imgs, masks in tqdm(train_loader):
        imgs, masks = imgs.to(DEVICE), masks.to(DEVICE)

        optimizer.zero_grad()
        preds = model(imgs)
        loss = loss_fn(preds, masks)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    model.eval()
    val_iou = 0

    with torch.no_grad():
        for imgs, masks in val_loader:
            imgs, masks = imgs.to(DEVICE), masks.to(DEVICE)
            preds = model(imgs)
            val_iou += iou_score(preds, masks).item()

    val_iou /= len(val_loader)

    print(f"Epoch [{epoch+1}/{EPOCHS}] | Train Loss: {train_loss:.4f} | Val IoU: {val_iou:.4f}")

    if val_iou > best_iou:
        best_iou = val_iou
        torch.save(model.state_dict(), "best_unet_sos.pth")
        print("Saved best model")

100%|██████████| 807/807 [03:15<00:00,  4.12it/s]


Epoch [1/80] | Train Loss: 232.3441 | Val IoU: 0.7225
Saved best model


100%|██████████| 807/807 [02:17<00:00,  5.85it/s]


Epoch [2/80] | Train Loss: 184.7518 | Val IoU: 0.7285
Saved best model


100%|██████████| 807/807 [02:16<00:00,  5.93it/s]


...

Epoch [78/80] | Train Loss: 126.2185 | Val IoU: 0.7764


100%|██████████| 807/807 [02:46<00:00,  4.84it/s]


Epoch [79/80] | Train Loss: 123.2594 | Val IoU: 0.7791


100%|██████████| 807/807 [02:33<00:00,  5.25it/s]


Epoch [80/80] | Train Loss: 127.4512 | Val IoU: 0.7735
